In [2]:
import os

In [3]:
%pwd

'c:\\Users\\vishw\\Documents\\cap_project_tulasi\\YOLO-missile-detection-capstone-project\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\vishw\\Documents\\cap_project_tulasi\\YOLO-missile-detection-capstone-project'

In [10]:
from dataclasses import dataclass

@dataclass
class ModelPredictionConfig:
  trained_weights_path: str
  test_images_path: str
  results_path: str
  confidence_threshold: float 


In [11]:
from dataclasses import dataclass
from YOLOv8_objectdetection.constants.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from YOLOv8_objectdetection.utils.common import read_yaml, create_directories
from YOLOv8_objectdetection import logger

In [12]:
class ConfigurationManager:
    def __init__(self):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)

    def get_model_prediction_config(self):
        # Assuming your params.yaml structure and keys, with added handling for missing keys
        confidence_threshold = self.params.get('prediction_params', {}).get('confidence_threshold', 0.25)  # Provide default value
        
        # Assuming 'model_prediction' is correctly loaded from your config
        config = self.config['model_prediction']

        return ModelPredictionConfig(
            trained_weights_path=config.get('trained_weights_path'),
            test_images_path=config.get('test_images_path'),
            results_path=config.get('results_path'),
            confidence_threshold=confidence_threshold,
        )


In [13]:
from dataclasses import dataclass
from YOLOv8_objectdetection.constants.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from YOLOv8_objectdetection.utils.common import read_yaml, create_directories
from YOLOv8_objectdetection import logger
from pathlib import Path
from ultralytics import YOLO
import os

class YOLOInference:
    def __init__(self, config: ModelPredictionConfig):
        self.config = config
        self.model = YOLO(self.config.trained_weights_path)
        # Assume default confidence threshold if not specified in params
        self.confidence_threshold = self.config.confidence_threshold

    def run_inference(self):
        # Ensure test_images_path directory exists
        assert os.path.isdir(self.config.test_images_path), f"Directory does not exist: {self.config.test_images_path}"
        
        # Collect image file paths
        image_paths = [os.path.join(self.config.test_images_path, image_file) for image_file in os.listdir(self.config.test_images_path) if image_file.endswith(('.jpg', '.png', '.jpeg'))]  # Ensure these are files
        
        # Ensure results_path directory exists or create it
        Path(self.config.results_path).mkdir(parents=True, exist_ok=True)
        
        results = self.model(image_paths, conf=self.confidence_threshold)  # Assuming this correctly handles a list of image file paths
        
        for idx, result in enumerate(results):
            original_filename = os.path.basename(image_paths[idx])
            result_filename = f"{original_filename.rsplit('.', 1)[0]}_result.jpg"
            result_path = os.path.join(self.config.results_path, result_filename)
            
            # Assuming result has a method to save to a file path
            result.save(result_path)
            print(f"Result saved to {result_path}")

if __name__ == "__main__":
    config_manager = ConfigurationManager()
    model_prediction_config = config_manager.get_model_prediction_config()

    # Make sure directories exist or are created
    create_directories([model_prediction_config.results_path])

    inference = YOLOInference(model_prediction_config)
    inference.run_inference()



[2024-02-27 15:09:14,837: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-27 15:09:14,841: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-27 15:09:14,842: INFO: common: created directory at: test/test_results]

0: 640x640 (no detections), 1: 640x640 (no detections), 2: 640x640 1 Missile, 3: 640x640 4 Missiles, 431.9ms
Speed: 5.2ms preprocess, 108.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Result saved to test/test_results\car_test_image_result.jpg
Result saved to test/test_results\test_image_02_result.jpg
Result saved to test/test_results\test_image_03_result.jpg
Result saved to test/test_results\test_image_1_result.jpg
